In [3]:
# Importing Libraries
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et

In [6]:
# Parsing
tree=et.parse('terkini.xml')
root=tree.getroot()

In [7]:
# Empty list of each data that will be filled
Title = []
Link = []
Timestamp = []
Detail_article = []

In [11]:
# Reading
for title in root.iter('title'):
    Title.append(title.text)
    
for url in root.iter('link'):
    Link.append(url.text)
    
for pubdate in root.iter('pubDate'):
    Timestamp.append(pubdate.text)
    
for detail in root.iter('description'):
    Detail_article.append(detail.text)

In [9]:
# Data in dataframe
News_detail_df = pd.DataFrame(list(zip(Title, Link, Timestamp, Detail_article)), 
                       columns=['Title', 'Link', 'Timestamp', 'Detail_article'])

In [12]:
# top 5 data scrapped
News_detail_df.head()

,Title,Link,Timestamp,Detail_article
0,ANTARA News Jambi Terkini,https://jambi.antaranews.com,"Sun, 28 Aug 2022 21:39:36 +0700",News And Service from ANTARA News Jambi
1,Malisya Deslilian lulus Fakultas Kedokteran Un...,https://jambi.antaranews.com/berita/519381/mal...,"Sun, 28 Aug 2022 21:22:25 +0700","<img src=""https://img.antaranews.com/cache/255..."
2,Polda Jambi perkuat pembinaan sikap personel u...,https://jambi.antaranews.com/berita/519377/pol...,"Sun, 28 Aug 2022 20:42:00 +0700","<img src=""https://img.antaranews.com/cache/255..."
3,Ribuan peserta antusias ikuti Jambi Color Run ...,https://jambi.antaranews.com/berita/519373/rib...,"Sun, 28 Aug 2022 20:27:58 +0700","<img src=""https://img.antaranews.com/cache/255..."
4,Pertamina Patra Niaga Regional Sumbagsel sesal...,https://jambi.antaranews.com/berita/519369/per...,"Sun, 28 Aug 2022 19:18:58 +0700","<img src=""https://img.antaranews.com/cache/255..."
